In [37]:
import numpy as np

datain = np.load('RFC4LBGDW_input_data.npy',allow_pickle=True).item()
print(f'# The input file contains a dictionary with two keys, {list(datain.keys())}.')
dva = datain['dva']; Tr = datain['transmission']
print(f'# \'transmission\' brings another dict object containing the main transmission data for five redshifts, z=6, 6.5, 7, 8, & 10. ')
print(f'# The data for each redshift can be accessed using another key, {list(Tr.keys())}, where 6h means 6.5.')
print(f'# ')

print(dva.shape)
print(Tr['6'].shape)


# The input file contains a dictionary with two keys, ['transmission', 'dva'].
# 'transmission' brings another dict object containing the main transmission data for five redshifts, z=6, 6.5, 7, 8, & 10. 
# The data for each redshift can be accessed using another key, ['6h', '7', '8', '10', '6'], where 6h means 6.5.
# 
(100,)
(203796, 100)
